In [1]:
import pandas as pd
import os
import json

# Read input data

In [2]:
filename = "Results_Adora2a.txt"

In [3]:
with open(filename) as file:
    lines = file.readlines()

In [4]:
results_line = False
header_read = False #second =
exp_cnt = 0 # every new #
conf_cnt = 0
new_experiment = True
bs_sel = None
selection = None
begin_count = False
experiments = {"id":[], "dimred":[], "feat":[], "clust":[], "thr": [], "conf_cnt": [], "conf_cnt_pre_thr": []}
experiment_frame = {"exp_id":[], "frame_id":[]}

i=0

while i < len(lines):
    line = lines[i]
    i+=1
    if "RESULTS" in line:
        results_line = True
        continue
    if results_line == True and "Binding site:" in line:
        bs_sel = line
        continue
    if results_line == True and "Selection:" in line:
        selection = line
        continue
    if results_line == True and not bs_sel is None and line[0]=="=" and not header_read:
        header_read = True
        new_exp = True
        continue
    if  header_read == True:
        if new_experiment ==True and "Data reduction: " in line:
            print(line)
            new_experiment = False
            experiments["id"].append(exp_cnt)
            experiments["dimred"].append(line[len("Data reduction: "):-1])
            line = lines[i]
            i+=1
            experiments["feat"].append(line[len("Featurization: "):-1])
            line = lines[i]
            i+=1
            experiments["clust"].append(line[len("Clustering: "):-1])
            line = lines[i]
            i+=1
            line = lines[i]
            i+=1
            if "threshold" in line:
                
                experiments["thr"].append(line.split()[4])
                experiments["conf_cnt_pre_thr"].append(int(line.split()[0]))
                line = lines[i]
                i+=1
            else:
                experiments["thr"].append("-")
                experiments["conf_cnt_pre_thr"].append("no thr")
            begin_count = True
            continue
            
    if begin_count == True:
        if "Closest conformation" in line:
            frame_id = int(line.split()[6])
            experiment_frame["exp_id"].append(exp_cnt)
            experiment_frame["frame_id"].append(frame_id)
            conf_cnt+=1
            continue
            
        if "==" in line:
            experiments["conf_cnt"].append(conf_cnt)
            conf_cnt = 0
            exp_cnt += 1
            begin_count = False
            new_experiment = True
            continue


Data reduction: PCA

Data reduction: PCA

Data reduction: PCA

Data reduction: PCA

Data reduction: TICA

Data reduction: TICA

Data reduction: TICA

Data reduction: TICA

Data reduction: HDE

Data reduction: HDE

Data reduction: HDE

Data reduction: HDE



In [5]:
exp_frame_df = pd.DataFrame(experiment_frame)

In [9]:
exp_frame_df.to_csv("experiment_selframe.csv")

In [7]:
exp_df = pd.DataFrame(experiments)

In [11]:
exp_df.to_csv("experiment_metadata.csv")

In [8]:
exp_df

,id,dimred,feat,clust,thr,conf_cnt,conf_cnt_pre_thr
0,0,PCA,residue_COM&distances_ca,Elbow (2-100) - Kmeans,0.005,14,18
1,1,PCA,residue_COM&distances_ca,Silhouette - Kmeans,-,2,no thr
2,2,PCA,residue_COM&distances_ca,Elbow (2-100) - GMMs,0.005,6,6
3,3,PCA,residue_COM&distances_ca,Silhouette - GMMs,-,2,no thr
4,4,TICA,residue_COM&distances_ca,Elbow (2-100) - Kmeans,0.005,11,27
5,5,TICA,residue_COM&distances_ca,Silhouette - Kmeans,-,3,no thr
6,6,TICA,residue_COM&distances_ca,Elbow (2-100) - GMMs,0.005,4,26
7,7,TICA,residue_COM&distances_ca,Silhouette - GMMs,-,3,no thr
8,8,HDE,residue_COM&distances_ca,Elbow (2-100) - Kmeans,0.005,1,28
9,9,HDE,residue_COM&distances_ca,Silhouette - Kmeans,-,2,no thr


In [100]:
exp_frame_df[exp_frame_df["exp_id"].isin([3])].sort_values(by="frame_id")

,exp_id,frame_id
23,3,154
22,3,1229


In [101]:
exp_frame_df[exp_frame_df["exp_id"].isin([7])].sort_values(by="frame_id")

,exp_id,frame_id
42,7,858
44,7,1107
43,7,1590


In [102]:
exp_frame_df[exp_frame_df["exp_id"].isin([11])].sort_values(by="frame_id")

,exp_id,frame_id
49,11,858
48,11,1419


# Prepare mapping

In [111]:
#--------------------------------
# map pdb names to trajectory frames

# they were arranged in order of ascending indexes
# trajectory indexes start from 0 - defined in clusten.py choose conf with argwhere

with open("pdb_file_names.json") as fp:
    file_names = json.load(fp)
    
def get_file_ind(fname): 
    return int(fname[len("at-frame"):fname.find("_")])

file_names.sort(key = get_file_ind)

dict_map = {"traj_ind": [], "file_name": [], "file_index":[]}

for i, fname in enumerate(file_names): 
    f_num = int(fname[len("at-frame"):fname.find("_")])
    dict_map["file_name"].append(fname)
    dict_map["file_index"].append(f_num)
    dict_map["traj_ind"].append(i)
    

In [114]:
pd.DataFrame(dict_map).to_csv("traj_frame_file_index_map.csv", index = False)

In [115]:
pd.read_csv("traj_frame_file_index_map.csv")

,traj_ind,file_name,file_index
0,0,at-frame0_noH.pdb,0
1,1,at-frame200_noH.pdb,200
2,2,at-frame400_noH.pdb,400
3,3,at-frame600_noH.pdb,600
4,4,at-frame800_noH.pdb,800
...,...,...,...
2995,2995,at-frame623600_noH.pdb,623600
2996,2996,at-frame623800_noH.pdb,623800
2997,2997,at-frame624000_noH.pdb,624000
2998,2998,at-frame624200_noH.pdb,624200
